## Загрузка предобученной модели в блокнот

#### Подключение к собственному Google Drive

In [ ]:
"""
Для воспроизведения кода на Вашем компьютере, 
запускать ячейку НЕ НУЖНО. Достаточно загрузить 
в локальное хранилище блокнота архив saved_model_trt.zip
"""

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
"""
В случае запуска с Вашего устройства,
раскомментировать следующую строчку и 
запустить выполнение данной ячейки
"""

# !unzip saved_model_trt.zip

#### Импорт функционала по предобработке изображения для сети VGG19

In [ ]:
from tensorflow.keras.applications.vgg19 import preprocess_input

In [ ]:
import tensorflow as tf

"""
Закомментировать следующую строчку, в случае запуска на Вашем устройстве
"""

model_trt = tf.keras.models.load_model('/content/gdrive/MyDrive/models/Diploma ML-mid (default)/vgg19/saved_model_trt')

"""
В случае запуска на Вашем устройстве, уберите комментарий 
со следующих строк, чтобы загрузить модель в локальное хранилише 
"""

# !unzip saved_model_trt.zip
# model_trt = tf.keras.models.load_model('saved_model_trt')

import numpy as np

#Warm-up
_ = model_trt(np.zeros((1, 128, 128, 3), dtype=np.float32))

## Чтение данных для предсказаний 

#### Функция для чтения изображений из папки и записи в словарь как "имя файла": "массив изображения"

In [ ]:
import cv2
import os

def load_images_from_folder(folder):
    images = {}
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images[filename] = img
    return images

#### Загрузка изображений для тестирования модели

In [ ]:
"""
Закомментировать следующую строчку, в случае запуска на Вашем устройстве
"""

!unzip gdrive/My\ Drive/data/Diploma\ ML-mid/test_kaggle.zip

"""
Для воспроизведения кода на Вашем компьютере, 
убрать комментарий со следующей строчки. 
Достаточно загрузить в локальное хранилище 
блокнота архив test_kaggle.zip
"""

# !unzip test_kaggle.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: test_kaggle/4979.jpg    
  inflating: test_kaggle/4965.jpg    
  inflating: test_kaggle/4989.jpg    
  inflating: test_kaggle/4967.jpg    
  inflating: test_kaggle/4976.jpg    
  inflating: test_kaggle/4963.jpg    
  inflating: test_kaggle/4964.jpg    
  inflating: test_kaggle/4994.jpg    
  inflating: test_kaggle/4995.jpg    
  inflating: test_kaggle/4996.jpg    
  inflating: test_kaggle/4987.jpg    
  inflating: test_kaggle/4968.jpg    
  inflating: test_kaggle/4971.jpg    
  inflating: test_kaggle/4990.jpg    
  inflating: test_kaggle/4972.jpg    
  inflating: test_kaggle/4982.jpg    
  inflating: test_kaggle/4962.jpg    
  inflating: test_kaggle/4992.jpg    
  inflating: test_kaggle/4985.jpg    
  inflating: test_kaggle/4974.jpg    
  inflating: test_kaggle/4977.jpg    
  inflating: test_kaggle/4940.jpg    
  inflating: test_kaggle/4948.jpg    
  inflating: test_kaggle/4944.jpg    
  inflating: test_ka

In [ ]:
images = load_images_from_folder("test_kaggle")

In [ ]:
names = images.keys()

## Классификация эмоций 

#### Заполняем словарь в виде "название файла": "предсказание"

In [ ]:
for name in names:
  image = images[name]
  data = np.empty((1, 128, 128, 3))
  image = cv2.resize(image, (128, 128))
  data[0] = image
  data = preprocess_input(data).astype('float32')
  prediction = model_trt(data)
  images[name] = np.argmax(prediction)

In [ ]:
import pandas as pd

df = pd.DataFrame({"image_path": names, "emotion": images.values()})
df

,image_path,emotion
0,4579.jpg,7
1,1105.jpg,3
2,3998.jpg,1
3,2044.jpg,6
4,2361.jpg,3
...,...,...
5013,4812.jpg,3
5014,4719.jpg,0
5015,4861.jpg,2
5016,2108.jpg,5


## Офрматирование датафрейма для создания посылки на Kaggle

#### В течение исследований выяснилось, что в папке test_kaggle имеются дубликаты, которые именуются как "название (1)", "название (2)" и т.д. Чтобы избавиться от них, ставим метку напротив строчек, где в названии файла имеется символ скобки '(' 

In [ ]:
df["check"] = 1
for i in range(len(df)):
  if "(" in df.image_path[i]:
    df.check[i] = 0  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
df

,image_path,emotion,check
0,4579.jpg,7,1
1,1105.jpg,3,1
2,3998.jpg,1,1
3,2044.jpg,6,1
4,2361.jpg,3,1
...,...,...,...
5013,4812.jpg,3,1
5014,4719.jpg,0,1
5015,4861.jpg,2,1
5016,2108.jpg,5,1


#### Избавимся от дубликатов

In [ ]:
fix_df = df[df.check == 1]

In [ ]:
fix_df.sort_values("image_path", inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
fix_df

,image_path,emotion,check
4704,0.jpg,6,1
1248,1.jpg,5,1
2399,10.jpg,3,1
3875,100.jpg,7,1
1561,1000.jpg,7,1
...,...,...,...
1547,995.jpg,7,1
4617,996.jpg,4,1
5003,997.jpg,0,1
3022,998.jpg,7,1


#### Просо отсортировать колонку image_path нельзя. Нужно обработать названия файлав иначе. Запишем в новую колонку всё, что идёт в столбце image_path до символа точки '.'

In [ ]:
fix_df["name"] = "null"
fix_df["name"] = fix_df["image_path"].apply(lambda x: x.split(".")[0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Изменим тип получившейся колонки в численный и отсортируем 

In [ ]:
fix_df["name"]
fix_df["name"] = fix_df["name"].apply(pd.to_numeric)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
fix_df.sort_values("name", inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### Избавимся от второстепенных колонок 

In [ ]:
fix_df.drop(columns=['name', 'check'], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


#### Так как ответы модели - индексы классов, а файл для Kaggle должен включать в себя непосредственно названия эмоций

In [ ]:
emotions = {8: "uncertain", 7: "surprise", 6: "sad", 5: "neutral", 4: "happy", 3: "fear", 2: "disgust", 1: "contempt", 0: "anger"}

fix_df["emotion"] = fix_df["emotion"].apply(lambda x: emotions[x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


#### Установим новый столбец для индексов

In [ ]:
fix_df.set_index("image_path", inplace=True)

## Сохранение датафрейма в файл .csv

#### Формируем файл для отправки в систему

In [ ]:
fix_df.to_csv("submission.csv")